## Import

In [1]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("rec").getOrCreate()

## Load data

We use movie dataset to build a recommendation system

In [2]:
import os

os.listdir()

['.ipynb_checkpoints',
 'Consulting Project - Recommender Systems.ipynb',
 'Meal_Info.csv',
 'movielens_ratings.csv',
 'My_code_along_recommend.ipynb',
 'Recommender_Code_Along.ipynb']

In [4]:
df = spark.read.csv("movielens_ratings.csv", inferSchema=True, header=True)

In [5]:
df.show(5)

+-------+------+------+
|movieId|rating|userId|
+-------+------+------+
|      2|   3.0|     0|
|      3|   1.0|     0|
|      5|   2.0|     0|
|      9|   4.0|     0|
|     11|   1.0|     0|
+-------+------+------+
only showing top 5 rows



In [6]:
print(df.count(), len(df.columns))

1501 3


In [7]:
df.describe().show()

+-------+------------------+------------------+------------------+
|summary|           movieId|            rating|            userId|
+-------+------------------+------------------+------------------+
|  count|              1501|              1501|              1501|
|   mean| 49.40572951365756|1.7741505662891406|14.383744170552964|
| stddev|28.937034065088994| 1.187276166124803| 8.591040424293272|
|    min|                 0|               1.0|                 0|
|    max|                99|               5.0|                29|
+-------+------------------+------------------+------------------+



In [8]:
df.printSchema()

root
 |-- movieId: integer (nullable = true)
 |-- rating: double (nullable = true)
 |-- userId: integer (nullable = true)



In [9]:
train, test = df.randomSplit([0.8, 0.2])

## Building recommender system

In [10]:
from pyspark.ml.recommendation import ALS
from pyspark.ml.evaluation import RegressionEvaluator

In [11]:
# create an install
als = ALS(userCol="userId", itemCol="movieId", ratingCol="rating", maxIter=5, regParam=0.01)

In [12]:
model = als.fit(train)

In [13]:
# prediction
pred = model.transform(test)

In [14]:
pred.show()

+-------+------+------+-----------+
|movieId|rating|userId| prediction|
+-------+------+------+-----------+
|     85|   1.0|     2|-0.28040802|
|     65|   2.0|     3|  0.7268968|
|     53|   1.0|     7|  4.1707096|
|     53|   1.0|    25| 0.92407835|
|     78|   1.0|     8|  0.7752889|
|     34|   3.0|    25|-0.21652365|
|     34|   1.0|    14|  0.8549364|
|     81|   1.0|    16| -2.6524897|
|     81|   1.0|     3|  0.9886235|
|     28|   1.0|    14|  2.3720064|
|     76|   1.0|    25|  1.6710465|
|     26|   1.0|     3|   3.602425|
|     26|   1.0|    19|   2.165067|
|     26|   1.0|     2|  1.5116813|
|     27|   1.0|     5|    4.86423|
|     44|   2.0|    12| 0.29755223|
|     44|   1.0|    22|  -2.184136|
|     44|   1.0|     8| -1.4975076|
|     44|   1.0|    14|  2.3715377|
|     12|   1.0|    16|  -2.817724|
+-------+------+------+-----------+
only showing top 20 rows



By observing we should see that our rec. system works not so well. That is understandable given very small dataset for rec. engine

## Evaluation

In [16]:
myeval = RegressionEvaluator(predictionCol="prediction", labelCol="rating", metricName="rmse")

In [18]:
myerr = myeval.evaluate(pred)

In [19]:
myerr

1.8585514170582034

## A single user

In [20]:
single_user = test.filter(test["userId"]==11).select(["movieId", "userId"])

In [21]:
single_user.show()

+-------+------+
|movieId|userId|
+-------+------+
|      0|    11|
|     18|    11|
|     22|    11|
|     35|    11|
|     41|    11|
|     43|    11|
|     45|    11|
|     69|    11|
|     79|    11|
|     90|    11|
|     94|    11|
|     97|    11|
+-------+------+



In [23]:
# for predicting we just need movie id and user id
single_pred = model.transform(single_user)

In [24]:
single_pred.show(3)

+-------+------+----------+
|movieId|userId|prediction|
+-------+------+----------+
|     22|    11|  1.136384|
|     94|    11|  1.153231|
|     41|    11|-0.8460221|
+-------+------+----------+
only showing top 3 rows



In [25]:
single_pred.orderBy("prediction", ascending=False).show()

+-------+------+-----------+
|movieId|userId| prediction|
+-------+------+-----------+
|     18|    11|  4.9206147|
|     90|    11|   3.066122|
|      0|    11|  1.7394315|
|     79|    11|  1.6466572|
|     94|    11|   1.153231|
|     22|    11|   1.136384|
|     69|    11|   0.885702|
|     43|    11| 0.35909772|
|     35|    11|-0.08083272|
|     97|    11|-0.30747512|
|     45|    11|-0.64249897|
|     41|    11| -0.8460221|
+-------+------+-----------+



So if we don't have the rating, we would recommend user 11 movie number 18, and then movie number 90